In [1]:
from osm_integration import OSMManager
from models.Adult import Adult
from datetime import datetime
import folium

In [2]:
# Define the center point and radius for OSMManager
center_point = (10.8231, 106.6297)  # Example: Ho Chi Minh City center
radius = 10000  # 10 km radius

# Initialize OSMManager
osm_manager = OSMManager(center_point, radius, network_type="drive")

Loading graph for center point (10.8231, 106.6297) with radius 10000 meters...
Scanning for residential...
Scanning for residential near (10.8231, 106.6297) within 10000 meters...
Found 301 residential.
Scanning for parks...
Scanning for parks near (10.8231, 106.6297) within 10000 meters...
Found 466 parks.
Scanning for schools...
Scanning for schools near (10.8231, 106.6297) within 10000 meters...
Found 774 schools.
Scanning for workplaces...
Scanning for workplaces near (10.8231, 106.6297) within 10000 meters...
Found 517 workplaces.
Scanning for markets...
Scanning for markets near (10.8231, 106.6297) within 10000 meters...
Found 239 markets.
Scanning for healthcare...
Scanning for healthcare near (10.8231, 106.6297) within 10000 meters...
Found 459 healthcare.
Scanning for play_areas...
Scanning for play_areas near (10.8231, 106.6297) within 10000 meters...
Found 37 play_areas.
Scanning for gyms...
Scanning for gyms near (10.8231, 106.6297) within 10000 meters...
Found 37 gyms.


In [3]:
# Create an adult person
adult = Adult(unique_id=1, type="adult", speed=1.4, osm_manager=osm_manager)

Person 1 assigned home at (10.845788056566933, 106.61751965118172).
Person 1 assigned workplace at (10.783373744958267, 106.70171622756372).
Person 1 assigned gym at (10.8401535, 106.6531776).
Person 1 assigned market at (10.859631978268704, 106.61728235315348).
Trajectory built from (10.845788056566933, 106.61751965118172) to (10.783373744958267, 106.70171622756372) with 127 nodes.
Trajectory built from (10.783373744958267, 106.70171622756372) to (10.859631978268704, 106.61728235315348) with 143 nodes.
Trajectory built from (10.859631978268704, 106.61728235315348) to (10.783373744958267, 106.70171622756372) with 142 nodes.
Trajectory built from (10.783373744958267, 106.70171622756372) to (10.8401535, 106.6531776) with 108 nodes.
Trajectory built from (10.8401535, 106.6531776) to (10.845788056566933, 106.61751965118172) with 59 nodes.


In [4]:
# Print the assigned waypoints
print("\n--- Assigned Waypoints ---")
for waypoint, coords in adult.waypoints.items():
    if coords:
        print(f"{waypoint.capitalize()}: {coords}")


--- Assigned Waypoints ---
Home: (10.845788056566933, 106.61751965118172)
Workplace: (10.783373744958267, 106.70171622756372)
Market: (10.859631978268704, 106.61728235315348)
Gym: (10.8401535, 106.6531776)


In [5]:
# Print the schedule
print("\n--- Schedule ---")
for movement in adult.schedule:
    print(f"{movement['start_waypoint']} → {movement['end_waypoint']} at {movement['start_time']}")


--- Schedule ---
home → workplace at 07:00:00
workplace → market at 12:00:00
market → workplace at 13:00:00
workplace → gym at 18:00:00
gym → home at 20:00:00


In [6]:
# Print the detailed schedule with trajectories
print("\n--- Detailed Schedule ---")
for movement in adult.detail_schedule:
    print(f"{movement['start_waypoint']} → {movement['end_waypoint']}")
    print(f"  Start Time: {movement['start_time']}")
    print(f"  Arrival Time: {movement['arrival_time']}")
    print(f"  Distance: {movement['distance_m']} meters")
    print(f"  Travel Time: {movement['travel_time_s']} seconds")
    print(f"  Route Nodes: {movement['route_nodes']}")


--- Detailed Schedule ---
home → workplace
  Start Time: 07:00:00
  Arrival Time: 09:40:14.908484
  Distance: 13460.87187706115 meters
  Travel Time: 9614.908483615109 seconds
  Route Nodes: [7305453075, 5043145811, 7305453080, 4069486218, 8141356742, 8141356743, 1611021948, 8548211894, 8141356723, 8141356741, 8141297190, 2091954894, 3067835769, 2869810792, 2091954752, 2276515365, 2276493879, 5718947672, 5718945598, 6645630553, 11601639074, 2091954867, 2091954831, 2091954828, 2091954791, 2091954891, 11768103473, 3655114529, 3655114531, 5735879410, 2091954889, 2091954845, 5735332144, 2091954877, 11453123758, 2079964012, 2592269705, 2082378083, 11399978032, 2079964403, 2079964176, 11399978019, 366379261, 2079964288, 6631169239, 2079964221, 11414718179, 366467129, 2433201343, 2078950513, 2078950886, 2407526807, 366471478, 366426580, 2513642423, 5721686013, 366461736, 11323053473, 8995743761, 8995743760, 5762747868, 723876650, 5762825874, 366462099, 2081907930, 2293477528, 6252959053, 957

In [7]:
# Simulate position at a specific time
current_time = datetime.now().replace(hour=7, minute=30)  # Example time
position = adult.get_position_at_time(current_time)

print("\n--- Simulated Position ---")
print(f"Time: {current_time}")
print(f"Position (latitude, longitude): {position}")


--- Simulated Position ---
Time: 2025-01-12 07:30:39.286322
Position (latitude, longitude): (np.float64(10.82740681117545), np.float64(106.62493910173228))


In [8]:
# Create a Folium map centered at the person's home
home_coords = adult.waypoints["home"]
folium_map = folium.Map(location=home_coords, zoom_start=13)

# Add waypoints to the map
for waypoint, coords in adult.waypoints.items():
    if coords:
        folium.Marker(
            location=coords, 
            popup=waypoint.capitalize(), 
            icon=folium.Icon(color="blue")
        ).add_to(folium_map)

# Add trajectories to the map
for movement in adult.detail_schedule:
    route_nodes = movement["route_nodes"]
    route_coords = [(osm_manager.nodes.loc[node, "y"], osm_manager.nodes.loc[node, "x"]) for node in route_nodes]
    
    # Draw the route as a polyline
    folium.PolyLine(route_coords, color="red", weight=2.5, opacity=0.8).add_to(folium_map)

# Add current position to the map
folium.Marker(
    location=position,
    popup=f"Current Position at {current_time.time()}",
    icon=folium.Icon(color="green", icon="info-sign"),
).add_to(folium_map)

In [9]:
# Save the map to an HTML file
map_filename = "person_movement_map.html"
folium_map.save(map_filename)
print(f"Map saved to {map_filename}")


Map saved to person_movement_map.html
